### import packages

In [1]:
import keras

In [13]:
from keras.datasets import mnist

from keras import backend as K

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils

### hyperparameters

In [3]:
num_classes = 10
batch_size = 128
epochs = 12

### image properties

In [4]:
img_rows = 28
img_cols = 28

### load the datasets

In [5]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [6]:
y_test.shape

(10000,)

In [7]:
K.image_data_format()

'channels_last'

### data pre-processing

In [8]:
# setting conditions to reshape images, based on backend K setting

if K.image_data_format() is 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols, 1)
    input_shape = (1, img_rows, img_cols, 1)
    
# change the data type of trainig and test sets

X_train = X_train.astype('float32')
X_train = X_train / 255

X_test = X_test.astype('float32')
X_test = X_test / 255


In [9]:
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 1, 28, 28, 1)
60000 train samples
10000 test samples


In [17]:
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

### define model architecture

In [18]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

### train the model

In [19]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))

Epoch 1/12
469/469 [==============================] - 55s 24ms/step - loss: 2.2822 - accuracy: 0.1502 - val_loss: 2.1589 - val_accuracy: 0.5485
Epoch 2/12
469/469 [==============================] - 10s 22ms/step - loss: 2.1132 - accuracy: 0.4408 - val_loss: 1.8488 - val_accuracy: 0.7448
Epoch 3/12
469/469 [==============================] - 11s 22ms/step - loss: 1.7844 - accuracy: 0.6030 - val_loss: 1.3812 - val_accuracy: 0.7960
Epoch 4/12
469/469 [==============================] - 10s 22ms/step - loss: 1.3748 - accuracy: 0.6748 - val_loss: 0.9831 - val_accuracy: 0.8240
Epoch 5/12
469/469 [==============================] - 10s 22ms/step - loss: 1.0715 - accuracy: 0.7176 - val_loss: 0.7543 - val_accuracy: 0.8440
Epoch 6/12
469/469 [==============================] - 10s 22ms/step - loss: 0.8944 - accuracy: 0.7489 - val_loss: 0.6298 - val_accuracy: 0.8582
Epoch 7/12
469/469 [==============================] - 11s 22ms/step - loss: 0.7902 - accuracy: 0.7699 - val_loss: 0.5544 - val_accuracy:

### evaluate the model

In [21]:
score = model.evaluate(X_test, y_test, verbose=0)

In [22]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.4014960825443268
Test accuracy: 0.892799973487854


### save the model

In [23]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

### download files from colab

In [24]:
from google.colab import files
files.download("model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
files.download("model.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### load the weights whenever we want for future use

In [34]:
import tensorflow as tf

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = tf.keras.models.model_from_json(loaded_model_json)

loaded_model.load_weights('model.h5')

In [36]:
loaded_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
